In [ ]:

import pandas as pd
import numpy as np 
import re 
import nltk
nltk.download("punkt")
from nltk.corpus import stopwords 
sw=stopwords.words('english') 
from nltk.util import ngrams
from lib.config import connstr
from lib.utils import * 
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings('ignore') 
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR, SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression 


In [ ]:
gram_range=(2,3)
def normalizer(s):
    s=re.sub('[^a-z\s\-]','',s.lower())
    s=re.sub('\s{2,}',' ',s).strip()
    return s

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj) 
    
def vectorize(s, verbose=False):  
    s=normalize(s)
    if verbose: print(s)
    vect=CountVectorizer(ngram_range=gram_range , analyzer='word' )
    X = vect.fit_transform([s])
    d={}
    if verbose: print(vect.get_feature_names())
    for f in vectorizer.get_feature_names(): 
        d[f]=0
        if f in vect.get_feature_names():
            d[f]=X.toarray()[0][vect.get_feature_names().index(f)-1] 
    with open('out.txt' , 'w') as f:
        f.write(json.dumps(d, cls=NpEncoder, indent=2))
    return d     

vectorizer = CountVectorizer(ngram_range=gram_range, analyzer='word')

In [ ]:
df=sql_todf(f"""
    SELECT TOP 10000 PK_Question, FK_QuestionType, QuestionText
    FROM fsma_Questions WHERE FK_QuestionType IS NOT NULL AND QuestionText IS NOT NULL  
    AND FK_QuestionType IN (1,2,9,17,43) 
    AND PK_Question < 40000
    ORDER BY PK_QUESTION DESC
    """, connstr)   
df['NQuestionText']=df['QuestionText'].apply(normalizer) 
df['NQuestionText'].dropna(inplace=True)
    
df['vecttext']=df['NQuestionText'].apply(lambda s: s.upper())   
X = vectorizer.fit_transform(df['vecttext'])   
dff = pd.DataFrame(columns=vectorizer.get_feature_names(), data=X.toarray()) 
sparse_matrix=pd.merge(df, dff, left_index=True, right_index=True)
sparse_matrix
 

In [ ]:
def sm_predict(s,threshold=.25): 
    s=normalizer(s)
    print ( s )
    dff=sparse_matrix.loc[:, ['FK_QuestionType','NQuestionText']]
    for i,r in dff.iterrows():  
        ratio=SequenceMatcher(None, s, str(r['NQuestionText']) ).ratio() 
        dff.loc[i, 'ratio'] = ratio
        #if ratio > threshold: break
    d=dff.loc[dff.ratio > threshold]
    d=d.sort_values(by=['ratio'], ascending=False) 
    return d
dfp=sm_predict("""

    How many chickens are classified as high value
    """)
dfp


In [ ]:
dff=sparse_matrix.copy()
dff.drop(['NQuestionText', 'QuestionText', 'PK_Question', 'vecttext'], inplace=True, axis=1)
y = dff['FK_QuestionType'].values 
X = dff.drop(['FK_QuestionType' ], axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

model = LogisticRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
def predict_question_type(s):
    vectorized=vectorize(s)   
    prediction=model.predict([list( vectorized.values() )])
    return prediction[0]

In [72]:
prediction = predict_question_type(
    """
    How many chickens are classified as angelo
    """
)  
print ('PK_QuestionType : '+ str(prediction))


PK_QuestionType : 2
